In [20]:
### imports
import json
from nltk.tokenize import sent_tokenize, TweetTokenizer
from string import punctuation
import re
import regex
tokenizer = TweetTokenizer()
from nltk.corpus import stopwords
stopW = stopwords.words('english')
#nltk.download('stopwords')

In [21]:
### a method to Read a json file with img captions and article titles and parse it into set of documents



def getDocs(sourcePath):
    medDocs = []
    sourceFile = open(sourcePath, 'r')

    count = 1
    for line in sourceFile:
        medDocs.append(json.loads(line))
        count += 1

    sourceFile.close()
    
    # for k in range(1,count):
    #     print(str(k) + " : " + medDocs[k]['docId'])
    return medDocs


sourcePath = 'assets/medImages.jl'
medDocs = getDocs(sourcePath)
medDocs[262]

{'articleId': 'PMC7282154',
 'image_path': 'PMC7282154-6_i2376-0605-6-2-e50-f06.jpg',
 'image_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/bin/i2376-0605-6-2-e50-f06.jpg',
 'articleUrl': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/',
 'articleTitle': 'RECURRENT INVASIVE DUCTAL BREAST CARCINOMA PRESENTING AS PRIMARY ADRENAL INSUFFICIENCY WITH ADRENAL CRISIS',
 'caption': 'Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.'}

In [22]:
# method that cleans the title and caption text in each document
# the result is tokenized titles and caption without unwanted brackets, numbers, spaces, tags, unicode charts;
# result includes no punctuation and stopwards and tokens with length less than 2 symbols
# all tokens are lower case


from nltk.corpus import stopwords
stopW = stopwords.words('english')

def preprocessContent(content):
    sentences = sent_tokenize(content)
    tokens = []
    for sent in sentences:
        # removes unicode
        cleanedTokens = re.sub(r'[^\x00-\x7F]+', ' ', sent)
        # removes menions
        cleanedTokens = re.sub(r'@\w+', '', cleanedTokens)
        # removes numbers
        cleanedTokens = re.sub('[0-9]+', '', cleanedTokens)
        # removes nested brackets 
        cleanedTokens = regex.sub(r'\([^()]*+(?:(?R)[^()]*)*+\)', '', cleanedTokens) 
        # removes nested currly brackets
        cleanedTokens = regex.sub('\{(?:[^}{]|\{[^}{]*\})*\}', '', cleanedTokens) 
        # removes html tags
        cleanedTokens = re.sub('\<+/*\w*/*\>+', '', cleanedTokens)
        # removes punctuation
        cleanedTokens = re.sub(r'[%s]' % re.escape(punctuation), ' ', cleanedTokens)
        # removes doubled spaces
        cleanedTokens = re.sub(r'\s{2,}', ' ', cleanedTokens)
        sentTokens = tokenizer.tokenize(cleanedTokens)
        sentTokens = [tok.lower() for tok in sentTokens if not tok in stopW and len(tok) > 2]
        
        tokens += sentTokens
    return tokens


def cleanDocs(medDocs):
    cleanedDocs = medDocs
    for i, doc in enumerate(cleanedDocs):
        doc['articleTitle'] = preprocessContent(doc['articleTitle'] )
        doc['caption'] = preprocessContent(doc['caption'])

    return cleanedDocs

cleanedDocs = cleanDocs(medDocs)
print(len(cleanedDocs))
cleanedDocs[262]


263


{'articleId': 'PMC7282154',
 'image_path': 'PMC7282154-6_i2376-0605-6-2-e50-f06.jpg',
 'image_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/bin/i2376-0605-6-2-e50-f06.jpg',
 'articleUrl': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/',
 'articleTitle': ['recurrent',
  'invasive',
  'ductal',
  'breast',
  'carcinoma',
  'presenting',
  'primary',
  'adrenal',
  'insufficiency',
  'with',
  'adrenal',
  'crisis'],
 'caption': ['histologic',
  'section',
  'adrenal',
  'metastatic',
  'disease',
  'tumor',
  'cells',
  'arranged',
  'solid',
  'nests',
  'infiltrating',
  'lymph',
  'node',
  'tissue']}